<a href="https://colab.research.google.com/github/sunny9sinha/TSAI_Session_7/blob/main/Session7_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim

# from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator
import pandas as pd
import spacy
import numpy as np
import torchtext

import random
import math
import time

In [54]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [55]:
%%bash
python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [56]:
spacy_en = spacy.load('en_core_web_sm')

In [57]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [59]:
Question = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

Answer = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [102]:
s08_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cmuQA/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt", sep='\t',encoding='ISO-8859-1')
s09_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cmuQA/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt", sep='\t',encoding='ISO-8859-1')
s10_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/cmuQA/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt", sep='\t', quoting=3,encoding='ISO-8859-1')

In [103]:
print(s08_data.shape)
print(s09_data.shape)
print(s10_data.shape)

(1715, 6)
(825, 6)
(1458, 6)


In [104]:
print(s08_data.columns)
print(s09_data.columns)
print(s10_data.columns)

Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')
Index(['ArticleTitle', 'Question', 'Answer', 'DifficultyFromQuestioner',
       'DifficultyFromAnswerer', 'ArticleFile'],
      dtype='object')


In [105]:
s08_data = s08_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])
s09_data = s09_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])
s10_data = s10_data.drop(columns=['ArticleTitle','DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile'])

In [124]:
QA_data = pd.concat([s08_data,s09_data,s10_data], axis =0)
print(QA_data.head())
print(QA_data.shape)

                                            Question Answer
0  Was Abraham Lincoln the sixteenth President of...    yes
1  Was Abraham Lincoln the sixteenth President of...   Yes.
2  Did Lincoln sign the National Banking Act of 1...    yes
3  Did Lincoln sign the National Banking Act of 1...   Yes.
4                   Did his mother die of pneumonia?     no
(3998, 2)


In [110]:
def smallCase(data):
    data['Question'] = data['Question'].str.lower()
    data['Answer'] = data['Answer'].str.lower()
    return data

In [125]:
QA_data.Question = np.array(QA_data.Question).astype(str)
QA_data.Answer = np.array(QA_data.Answer).astype(str)
QA_data = smallCase(pd.DataFrame(QA_data))
print(QA_data.head())
print(QA_data.shape)

                                            Question Answer
0  was abraham lincoln the sixteenth president of...    yes
1  was abraham lincoln the sixteenth president of...   yes.
2  did lincoln sign the national banking act of 1...    yes
3  did lincoln sign the national banking act of 1...   yes.
4                   did his mother die of pneumonia?     no
(3998, 2)


In [126]:
from sklearn.model_selection import train_test_split

train, test_data = train_test_split(QA_data, test_size=0.3)
train = train.reset_index().drop(columns=['index'])
test_data = test_data.reset_index().drop(columns=['index'])
train_data, valid_data = train_test_split(train, test_size=0.25)
train_data = train_data.reset_index().drop(columns=['index'])
valid_data = valid_data.reset_index().drop(columns=['index'])

In [127]:
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

(2098, 2)
(700, 2)
(1200, 2)


In [129]:
fields = [('Question', Question),('Answer',Answer)]

In [130]:
train_data.shape

(2098, 2)

In [131]:
example = [torchtext.legacy.data.Example.fromlist([train_data.Question[i],train_data.Answer[i]], fields) for i in range(train_data.shape[0])] 

In [132]:
trainDataset = torchtext.legacy.data.Dataset(example, fields)

In [133]:
example = [torchtext.legacy.data.Example.fromlist([test_data.Question[i],test_data.Answer[i]], fields) for i in range(test_data.shape[0])] 

In [134]:
testDataset = torchtext.legacy.data.Dataset(example, fields)

In [135]:
example = [torchtext.legacy.data.Example.fromlist([valid_data.Question[i],valid_data.Answer[i]], fields) for i in range(valid_data.shape[0])] 

In [136]:
validDataset = torchtext.legacy.data.Dataset(example, fields)

In [137]:
print(f"Number of training examples: {len(trainDataset.examples)}")
print(f"Number of validation examples: {len(validDataset.examples)}")
print(f"Number of testing examples: {len(testDataset.examples)}")

Number of training examples: 2098
Number of validation examples: 700
Number of testing examples: 1200


In [138]:
print(vars(trainDataset.examples[0]))

{'Question': ['are', 'elephants', 'mammals', '?'], 'Answer': ['yes']}


In [139]:
Question.build_vocab(trainDataset)
Answer.build_vocab(trainDataset)

In [140]:
print(f"Unique tokens in Question vocabulary: {len(Question.vocab)}")
print(f"Unique tokens in Answer vocabulary: {len(Answer.vocab)}")

Unique tokens in Question vocabulary: 3545
Unique tokens in Answer vocabulary: 2387


In [141]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [142]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (trainDataset, validDataset, testDataset), 
    batch_size = BATCH_SIZE,sort=False, 
    device = device)

In [143]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [144]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [145]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        # print(trg.shape)
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [146]:
INPUT_DIM = len(Question.vocab)
OUTPUT_DIM = len(Answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [147]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(3545, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(2387, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=2387, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [148]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,099,539 trainable parameters


In [149]:
optimizer = optim.Adam(model.parameters())

In [150]:
TRG_PAD_IDX = Answer.vocab.stoi[Answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [151]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [152]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Question
            trg = batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [153]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [154]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 2s
	Train Loss: 5.929 | Train PPL: 375.834
	 Val. Loss: 5.238 |  Val. PPL: 188.334
Epoch: 02 | Time: 0m 2s
	Train Loss: 5.083 | Train PPL: 161.283
	 Val. Loss: 5.365 |  Val. PPL: 213.806
Epoch: 03 | Time: 0m 2s
	Train Loss: 4.963 | Train PPL: 143.016
	 Val. Loss: 5.439 |  Val. PPL: 230.202
Epoch: 04 | Time: 0m 2s
	Train Loss: 4.870 | Train PPL: 130.366
	 Val. Loss: 5.497 |  Val. PPL: 244.036
Epoch: 05 | Time: 0m 2s
	Train Loss: 4.783 | Train PPL: 119.492
	 Val. Loss: 5.579 |  Val. PPL: 264.817
Epoch: 06 | Time: 0m 2s
	Train Loss: 4.771 | Train PPL: 118.024
	 Val. Loss: 5.639 |  Val. PPL: 281.226
Epoch: 07 | Time: 0m 2s
	Train Loss: 4.640 | Train PPL: 103.537
	 Val. Loss: 5.662 |  Val. PPL: 287.791
Epoch: 08 | Time: 0m 2s
	Train Loss: 4.597 | Train PPL:  99.199
	 Val. Loss: 5.737 |  Val. PPL: 310.028
Epoch: 09 | Time: 0m 2s
	Train Loss: 4.550 | Train PPL:  94.644
	 Val. Loss: 5.805 |  Val. PPL: 331.919
Epoch: 10 | Time: 0m 2s
	Train Loss: 4.523 | Train PPL:  92.139


In [155]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.263 | Test PPL: 192.987 |
